In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth',1000)


## Open the .xml files, get the data I need, save it into a tuple, then save it into a .csv

In [2]:
try:
    general_tweets_corpus_train = pd.read_csv('/users/hnesman/Downloads/DATA_Science/TASS/general-tweets-train-tagged.csv', encoding='utf-8')
except:

    from lxml import objectify
    xml = objectify.parse(open('/users/hnesman/Downloads/DATA_Science/TASS/general-tweets-train-tagged.xml'))
    #sample tweet object
    root = xml.getroot()
    general_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity', 'agreement'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiments.polarity.value.text, tweet.sentiments.polarity.type.text]))
        row_s = pd.Series(row)
        row_s.name = i
        general_tweets_corpus_train = general_tweets_corpus_train.append(row_s)
    general_tweets_corpus_train.to_csv('/users/hnesman/Downloads/DATA_Science/TASS_Parsed/general-tweets-train-tagged.csv', index=False, encoding='utf-8')


In [64]:
len(general_tweets_corpus_train)

7219

In [65]:
try:
    stompol_tweets_corpus_train = pd.read_csv('/users/hnesman/Downloads/DATA_Science/TASS/stompol-tweets-train-tagged.csv', encoding='utf-8')
except:

    from lxml import objectify
    xml = objectify.parse(open('/users/hnesman/Downloads/DATA_Science/TASS/stompol-tweets-train-tagged.xml'))
    #sample tweet object
    root = xml.getroot()
    stompol_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [' '.join(list(tweet.itertext())), tweet.sentiment.get('polarity')]))
        row_s = pd.Series(row)
        row_s.name = i
        stompol_tweets_corpus_train = stompol_tweets_corpus_train.append(row_s)
    stompol_tweets_corpus_train.to_csv('/users/hnesman/Downloads/DATA_Science/TASS_Parsed/stompol-tweets-train-tagged.csv', index=False, encoding='utf-8')

In [66]:
len(stompol_tweets_corpus_train)

784

In [16]:
# cd /Users/hnesman/Downloads/DATA_Science/TASS
# !head -n 30 general-tweets-train-tagged.xml

/Users/hnesman/Downloads/DATA_Science/TASS


In [4]:
try:
    general_tweets_corpus_train = pd.read_csv('/users/hnesman/Downloads/DATA_Science/TASS/politics2013-tweets-test-tagged.csv', encoding='utf-8')
except:

    from lxml import objectify
    xml = objectify.parse(open('/users/hnesman/Downloads/DATA_Science/TASS/politics2013-tweets-test-tagged.xml'))
    #sample tweet object
    root = xml.getroot()
    pol_2013_tweets_corpus_train= pd.DataFrame(columns=('content', 'polarity', 'agreement'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiments.polarity.value.text, tweet.sentiments.polarity.type.text]))
        row_s = pd.Series(row)
        row_s.name = i
        pol_2013_tweets_corpus_train = pol_2013_tweets_corpus_train.append(row_s)
    pol_2013_tweets_corpus_train.to_csv('/users/hnesman/Downloads/DATA_Science/TASS_Parsed/politics2013-tweets-test-tagged.csv', index=False, encoding='utf-8')

In [67]:
len(pol_2013_tweets_corpus_train)

2500

In [5]:
tweets_corpus = pd.concat([
        pol_2013_tweets_corpus_train,
        stompol_tweets_corpus_train,
        general_tweets_corpus_train
        
    ])
tweets_corpus.head()

,agreement,content,polarity
0,AGREEMENT,"""@marianorajoy: En España las cosas se pueden, se deben y se van a hacer infinitamente mejor que estos últimos 4 años"" Eso son soluciones!!",P
1,DISAGREEMENT,"En PSO€ el que no corre vuela, todavía caliente el cadáver político de ZP y Rubalcaba y la PANtumaca buscando hueco #votaPP #sumatealcambio",N
2,AGREEMENT,#nomeolvido de q cuando se aprobo la ley del aborto libre todas tus ministras saltaban de alegria en el congreso y yo lloré @ConRubalcaba,N
3,AGREEMENT,#CCOO exige al nuevo Gobierno que reactive el mercado interno de automoción para relanzar al sector #20N : http://t.co/XScRUEKh via @AddThis,NEU
4,DISAGREEMENT,@marianorajoy A mí inviable me parecen los fraudes fiscales que cometen miembros de su partido no una ley que afecta a miles de familias.,P


# Building the Model

## Set up the labels of the variable 'Polarity', as I want to predict binary values, I have to transform some values of this variable

    1) I will set up my dataset 'tweets_corpus', I will drop all those rows with polarity = 'NEU' , 'NONE'
    2) I will join N with N+, into a new value = 0
    3) I will join P with P+, into a new value = 1

## Find a model capable to do a good perform with Tweets in Spanish (that's why I use TASS)

    1) Random Forest Classificator
    2) Logistic Regression

In [71]:
# I do an inspection of the values in the variable 'polarity'

print tweets_corpus.groupby('polarity').count()
tweets = tweets_corpus.copy()
tweets.columns
len(tweets)

          agreement  content
polarity                    
N              2033     2468
N+              847      847
NEU            1611     1797
NONE           1705     1705
P              1871     2034
P+             1652     1652


10503

In [69]:
# Create a cmap for the values of 'polarity'

cmap = {'P+': 1, 'P': 1,'NONE': 2,'NEU': 2, 'N': 0, 'N+': 0}
tweets['cpolarity'] = tweets.polarity.apply(lambda x: cmap[str(x)])
tweets.head()

,agreement,content,polarity,cpolarity
0,AGREEMENT,"""@marianorajoy: En España las cosas se pueden, se deben y se van a hacer infinitamente mejor que estos últimos 4 años"" Eso son soluciones!!",P,1
1,DISAGREEMENT,"En PSO€ el que no corre vuela, todavía caliente el cadáver político de ZP y Rubalcaba y la PANtumaca buscando hueco #votaPP #sumatealcambio",N,0
2,AGREEMENT,#nomeolvido de q cuando se aprobo la ley del aborto libre todas tus ministras saltaban de alegria en el congreso y yo lloré @ConRubalcaba,N,0
3,AGREEMENT,#CCOO exige al nuevo Gobierno que reactive el mercado interno de automoción para relanzar al sector #20N : http://t.co/XScRUEKh via @AddThis,NEU,2
4,DISAGREEMENT,@marianorajoy A mí inviable me parecen los fraudes fiscales que cometen miembros de su partido no una ley que afecta a miles de familias.,P,1


In [8]:
# Check everything run alright
print tweets.groupby('cpolarity').count()

           agreement  content  polarity
cpolarity                              
0               2880     3315      3315
1               3523     3686      3686
2               3316     3502      3502


In [9]:
# Drop all the tweets with 'cpolarity' = 2 (accordingly to the map are the 'None' or 'Neu' values)
data = tweets[tweets.cpolarity != 2]
data.cpolarity.value_counts(normalize=True)


1    0.526496
0    0.473504
Name: cpolarity, dtype: float64

In [72]:
# print data.groupby('cpolarity').count()
data['cpolarity'].dtype
len(data)

7001

Now my dataset is ready to train a model for Classification:

-I will try a Random Forest to predict the polarity of the Tweet, based on the Features of the Content
    - So I need to extract the Features, then fit the model, and see test the results
    - For CountVectorizer I am gonna use the same model I used in the MediaNews analysis

In [11]:
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer

/Users/hnesman/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
# I use the same list of Stop Words that I used for Argentina Tweets

# coding: utf8
from __future__ import unicode_literals


STOP_WORDS = set(""" 100 20 http https 'http co' 'https co' 'cont co' cont co
none anos ano año años actualmente acuerdo adelante ademas además adrede afirmó agregó ahi ahora ahí
al algo alguna algunas alguno algunos algún alli allí alrededor ambos ampleamos
antano antaño ante anterior antes apenas aproximadamente aquel aquella aquellas
aquello aquellos aqui aquél aquélla aquéllas aquéllos aquí arriba arribaabajo
aseguró asi así atras aun aunque ayer añadió aún
bajo bastante bien breve buen buena buenas bueno buenos
cada casi cerca cierta ciertas cierto ciertos cinco claro comentó como con
conmigo conocer conseguimos conseguir considera consideró consigo consigue
consiguen consigues contigo contra cosas creo cual cuales cualquier cuando
cuanta cuantas cuanto cuantos cuatro cuenta cuál cuáles cuándo cuánta cuántas
cuánto cuántos cómo
da dado dan dar de debajo debe deben debido decir dejó del delante demasiado
demás dentro deprisa desde despacio despues después detras detrás dia dias dice
dicen dicho dieron diferente diferentes dijeron dijo dio donde dos durante día
días dónde
ejemplo el ella ellas ello ellos embargo empleais emplean emplear empleas
empleo en encima encuentra enfrente enseguida entonces entre era eramos eran
eras eres es esa esas ese eso esos esta estaba estaban estado estados estais
estamos estan estar estará estas este esto estos estoy estuvo está están ex
excepto existe existen explicó expresó él ésa ésas ése ésos ésta éstas éste
éstos
fin final fue fuera fueron fui fuimos
general gran grandes gueno
ha haber habia habla hablan habló habrá había habían hace haceis hacemos hacen hacer
hacerlo haces hacia haciendo hago han hasta hay haya he hecho hemos hicieron
hizo horas hoy hubo
igual incluso indicó informo informó intenta intentais intentamos intentan
intentar intentas intento ir
junto
la lado largo las le lejos les llegó lleva llevar lo los luego lugar
mal manera manifestó mas mayor me mediante medio mejor mencionó menos menudo mi
mia mias mientras mio mios mis misma mismas mismo mismos modo momento mucha
muchas mucho muchos muy más mí mía mías mío míos
nada nadie ni ninguna ningunas ninguno ningunos ningún no nos nosotras nosotros
nuestra nuestras nuestro nuestros nueva nuevas nuevo nuevos nunca
ocho os otra otras otro otros
pais para parece parte partir pasada pasado paìs peor pero pesar poca pocas
poco pocos podeis podemos poder podria podriais podriamos podrian podrias podrá
podrán podría podrían poner por porque posible primer primera primero primeros
principalmente pronto propia propias propio propios proximo próximo próximos
pudo pueda puede pueden puedo pues
qeu que quedó queremos quien quienes quiere quiza quizas quizá quizás quién quiénes qué
raras realizado realizar realizó repente respecto
sabe sabeis sabemos saben saber sabes salvo se sea sean segun segunda segundo
según seis ser sera será serán sería señaló si sido siempre siendo siete sigue
siguiente sin sino sobre sois sola solamente solas solo solos somos son soy
soyos su supuesto sus suya suyas suyo sé sí sólo
tal tambien también tampoco tan tanto tarde te temprano tendrá tendrán teneis
tenemos tener tenga tengo tenido tenía tercera ti tiempo tiene tienen toda
todas todavia todavía todo todos total trabaja trabajais trabajamos trabajan
trabajar trabajas trabajo tras trata través tres tu tus tuvo tuya tuyas tuyo
tuyos tú
ultimo un una unas uno unos usa usais usamos usan usar usas uso usted ustedes
última últimas último últimos
va vais valor vamos van varias varios vaya veces ver verdad verdadera verdadero
vez vosotras vosotros voy vuestra vuestras vuestro vuestros
ya yo
""".split())

In [13]:
data.columns

Index([u'agreement', u'content', u'polarity', u'cpolarity'], dtype='object')

I will use CountVectorizer

In [56]:
content = data['content']
Cvectorizer = CountVectorizer(max_features = 100, 
                             ngram_range=(1, 2), 
                             stop_words= STOP_WORDS,
                             binary=False)

# Use `fit` to learn the vocabulary of the titles
Cvectorizer.fit(content)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = Cvectorizer.transform(content)
Cvectorizer.get_feature_names()
# X.shape
# print(X)

[u'000',
 u'20n',
 u'30',
 u'abrazo',
 u'agarzon',
 u'ahorapodemos',
 u'albert_rivera',
 u'alejandrosanz',
 u'amigos',
 u'andaluc\xeda',
 u'apoyo',
 u'cadavotovale',
 u'cambio',
 u'campa\xf1a',
 u'casa',
 u'cayo_lara',
 u'chac\xf3n',
 u'ciudadanoscs',
 u'congreso',
 u'conrubalcaba',
 u'crisis',
 u'david_busta',
 u'democracia',
 u'derechos',
 u'deuda',
 u'dinero',
 u'dl',
 u'domingo',
 u'd\xe9ficit',
 u'educaci\xf3n',
 u'elecciones',
 u'electoral',
 u'enhorabuena',
 u'equipo',
 u'espa\xf1a',
 u'espa\xf1oles',
 u'espero',
 u'eta',
 u'euros',
 u'falta',
 u'felicidades',
 u'feliz',
 u'ff',
 u'foto',
 u'gente',
 u'gobierno',
 u'gracias',
 u'gusta',
 u'huelga',
 u'impuestos',
 u'iu',
 u'iunida',
 u'javierarenas_pp',
 u'junta',
 u'justicia',
 u'laboral',
 u'ley',
 u'madrid',
 u'marianorajoy',
 u'mariviromero',
 u'ma\xf1ana',
 u'millones',
 u'ministro',
 u'mundo',
 u'noche',
 u'noches',
 u'nomeolvido',
 u'paro',
 u'partido',
 u'pa\xeds',
 u'pedroj_ramirez',
 u'personas',
 u'pol\xedtica',
 u'pp

I will use a Tf-idF for Word Vector

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
Tfvectorizer = TfidfVectorizer(max_df=0.5, max_features = 100,
                                 stop_words=STOP_WORDS)
X_TfidF = Tfvectorizer.fit_transform(content)
Tfvectorizer.get_feature_names()
X.shape

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [112]:
# Using 'vectorizer.get_feature_names' to adjust the STOP WORDS, I took off http https co cont
# feat = vectorizer.get_feature_names()
# feat

In [31]:
from sklearn.cross_validation import cross_val_score

model = RandomForestClassifier(n_estimators = 25)

# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(content)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(content).toarray()
y = data['cpolarity']


print type(X)
scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

<type 'numpy.ndarray'>
CV AUC [ 0.68500602  0.70804281  0.68439835], Average AUC 0.692482394466


In [47]:
model.fit(X, y)

all_feature_names = vectorizer.get_feature_names()
feature_importances = pd.DataFrame({'Features' : all_feature_names, 'Importance Score': model.feature_importances_})
feature_importances.sort_values('Importance Score', ascending=False).head()

,Features,Importance Score
46,gracias,0.065722
74,ppopular,0.022888
77,psoe,0.020824
51,iunida,0.020582
73,pp,0.020259


In [43]:
from sklearn import grid_search, cross_validation
from sklearn.model_selection import KFold

### I defined this above, this only for reference ###
# vectorizer.fit(content)
# X = vectorizer.transform(content).toarray()
# y = data['cpolarity']

# scores = cross_val_score(model, X, y, scoring='roc_auc') # tengo que ver cómo meto las Cross Validation dentro de GS

model = RandomForestClassifier(n_estimators = 5)

kf = KFold(n_splits=2)

gs = grid_search.GridSearchCV(estimator = model,
                             param_grid = {'n_estimators': [i for i in range(1,52,10)],
                                          "max_depth": [3, 5],
                                          "bootstrap": [True, False],
                                          "criterion": ["gini"]},
                             cv = cross_validation.KFold(n=len(X), n_folds=10), scoring='roc_auc')

gs.fit(X, y)
print gs.grid_scores_
print (gs.best_estimator_)
print (gs.best_score_)


[mean: 0.52187, std: 0.01844, params: {u'n_estimators': 1, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 3}, mean: 0.60861, std: 0.02824, params: {u'n_estimators': 11, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 3}, mean: 0.63720, std: 0.04157, params: {u'n_estimators': 21, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 3}, mean: 0.65228, std: 0.03877, params: {u'n_estimators': 31, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 3}, mean: 0.65206, std: 0.03929, params: {u'n_estimators': 41, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 3}, mean: 0.65398, std: 0.03606, params: {u'n_estimators': 51, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 3}, mean: 0.55220, std: 0.02513, params: {u'n_estimators': 1, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 5}, mean: 0.63408, std: 0.03896, params: {u'n_estimators': 11, u'bootstrap': True, u'criterion': u'gini', u'max_depth': 5}, mean: 0.65732, std: 0.04484, params: {u'n

Now I will try a Logistic Regression fit of my data, I will use Grid Search as well

In [44]:
from sklearn.linear_model import LogisticRegression

In [46]:
gs = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced']},
    cv=cross_validation.KFold(n=len(X), n_folds=10),
    scoring='roc_auc'
)

gs.fit(X, y)
print gs.grid_scores_
print (gs.best_estimator_)
print (gs.best_score_)

[mean: 0.70697, std: 0.04129, params: {u'C': 100000, u'class_weight': None}, mean: 0.70693, std: 0.04172, params: {u'C': 100000, u'class_weight': u'balanced'}, mean: 0.70697, std: 0.04129, params: {u'C': 10000, u'class_weight': None}, mean: 0.70693, std: 0.04172, params: {u'C': 10000, u'class_weight': u'balanced'}, mean: 0.70699, std: 0.04130, params: {u'C': 1000, u'class_weight': None}, mean: 0.70696, std: 0.04169, params: {u'C': 1000, u'class_weight': u'balanced'}, mean: 0.70712, std: 0.04103, params: {u'C': 100, u'class_weight': None}, mean: 0.70712, std: 0.04146, params: {u'C': 100, u'class_weight': u'balanced'}, mean: 0.70774, std: 0.03944, params: {u'C': 10, u'class_weight': None}, mean: 0.70766, std: 0.03995, params: {u'C': 10, u'class_weight': u'balanced'}, mean: 0.71009, std: 0.03673, params: {u'C': 1, u'class_weight': None}, mean: 0.71033, std: 0.03641, params: {u'C': 1, u'class_weight': u'balanced'}, mean: 0.70622, std: 0.03602, params: {u'C': 0.1, u'class_weight': None}, me